In [ ]:
import warnings
warnings.filterwarnings('ignore', 'numpy.dtype size changed')
warnings.filterwarnings( 'ignore', category=FutureWarning)

from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from pyproj import Proj
from scipy.ndimage import gaussian_filter
import xarray as xr

In [ ]:
date = datetime(2022, 1, 17, 12)

# Current Data
# ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p5deg_ana/TP')

# UCAR RDA Archive GFS Files 1 deg output
# ds = xr.open_dataset('https://rda.ucar.edu/thredds/dodsC/aggregations/g/ds083.2/2/TP')

# UCAR RDA Archive - Global 0.25 deg Data
ds = xr.open_dataset('https://rda.ucar.edu/thredds/dodsC/files/g/ds083.3/'
                     f'{date:%Y}/{date:%Y%m}/gdas1.fnl0p25.{date:%Y%m%d%H}.f00.grib2')

# Local File
#ds = xr.open_dataset('groundhogs_day_blizzard/GFS_{0:%Y%m%d}_{0:%H}00.nc'.format(date))

subset = dict(vertical=slice(100*units.hPa, None),
              time=date,
              lat=slice(60, 10),
              lon=slice(360-160, 360-50))

uwnd = ds['u-component_of_wind_isobaric'].metpy.sel(subset)
vwnd = ds['v-component_of_wind_isobaric'].metpy.sel(subset)
tmpk = ds['Temperature_isobaric'].metpy.sel(subset)
relh = ds['Relative_humidity_isobaric'].metpy.sel(subset)

lons, lats = np.meshgrid(uwnd.lon, uwnd.lat)

vtime = date

lev = uwnd.metpy.vertical

mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
datacrs = ccrs.PlateCarree()

# Transform Coordinates ahead of time
tlatlons = mapcrs.transform_points(ccrs.PlateCarree(), lons, lats)
clons = tlatlons[:,:,0]
clats = tlatlons[:,:,1]

isentlevs = list(range(285, 331, 3)) * units.kelvin

isent_anal = mpcalc.isentropic_interpolation_as_dataset(isentlevs, tmpk, relh, uwnd, vwnd)

In [ ]:
def plot_isentropic(level):
    print("Creating the {} K Level Maps".format(level))

    isentprs = mpcalc.smooth_n_point(isent_anal.pressure.sel(isentropic_level=level*units.K), 9, 5)
    isenttmp = mpcalc.smooth_n_point(isent_anal.temperature.sel(isentropic_level=level*units.K), 9, 5)
    isentrelh = mpcalc.smooth_n_point(isent_anal.Relative_humidity_isobaric.sel(isentropic_level=level*units.K), 9, 5)
    isentu = mpcalc.smooth_n_point(isent_anal['u-component_of_wind_isobaric'].sel(isentropic_level=level*units.K).metpy.convert_units('kt'), 9, 5)
    isentv = mpcalc.smooth_n_point(isent_anal['v-component_of_wind_isobaric'].sel(isentropic_level=level*units.K).metpy.convert_units('kt'), 9, 5)

    # For 1 deg data use 4
    # For 0.5 deg data use 5
    # For 0.25 deg data use 9
    wind_slice = (slice(None, None, 9), slice(None, None, 9))
    
    fig = plt.figure(1, figsize=(17,15))

    # 1st panel
    ax = plt.subplot(111, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cf = ax.contourf(clons, clats, isentrelh, range(70, 101, 1), cmap=plt.cm.Greens, norm=plt.Normalize(70,108))
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    cs = ax.contour(clons, clats, isentprs, range(50, 1001, 50), colors='black')
    plt.clabel(cs, fmt='%d')

    ax.barbs(lons[wind_slice], lats[wind_slice],
             isentu[wind_slice].values, isentv[wind_slice].values,
             transform=ccrs.PlateCarree())

    plt.title(f'{level} K Pressure (hPa), Relative Humidity (%), and Wind Barbs', loc='left')
    plt.title(f'Valid Time: {vtime}', loc='right')

    plt.savefig(f'Isentropic_{level}K_{date:%Y%m%d_%H}00.png', bbox_inches='tight', dpi=150)
    plt.show()
    plt.close()

In [ ]:
for level in [294, 297, 300, 303, 312, 315, 324, 330]:
    plot_isentropic(level)